# Survey Plan AI - Cloud Trainer (Git-Ops)

This notebook pulls the latest code from GitHub and trains on your Drive data.

In [ ]:
# 1. Mount Google Drive (Data & Results)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 2. Deploy Code (Git Clone/Pull)
import os
import shutil

REPO_URL = "https://github.com/14jadon14/survey-plan-AI"
REPO_NAME = REPO_URL.split("/")[-1].replace(".git", "")
CODE_DIR = f"/content/{REPO_NAME}"

if os.path.exists(CODE_DIR):
    print("\ud83d\udd04 Updating code...")
    !cd "{CODE_DIR}" && git pull
else:
    print("\ud83d\ude80 Cloning repository...")
    !git clone "{REPO_URL}"

print(f"\u2705 Code deployed to: {CODE_DIR}")

In [ ]:
# 3. Setup Environment & Data
# Install dependencies from the repo
!pip install -r "{CODE_DIR}/yolo_training/requirements.txt"

# Prepare Data (Fast IO: Copy zip from Drive -> Local VM)
# We assume data.zip is in 'My Drive/SurveyPlan AI/data.zip' OR just 'My Drive/data.zip'
DRIVE_ROOT = "/content/drive/MyDrive"
POSSIBLE_DATA_PATHS = [
    f"{DRIVE_ROOT}/SurveyPlan AI/data.zip",
    f"{DRIVE_ROOT}/data.zip"
]

DATA_ZIP = None
for p in POSSIBLE_DATA_PATHS:
    if os.path.exists(p):
        DATA_ZIP = p
        break

if DATA_ZIP:
    if not os.path.exists("/content/custom_data"):
        print(f"\ud83d\udce6 Found data at {DATA_ZIP}. Unzipping to local VM...")
        !unzip -q -o "{DATA_ZIP}" -d /content/custom_data
    else:
        print("\u2705 Data already ready.")
else:
    print("\u26a0\ufe0f Warning: data.zip not found in Drive. Ensure it is in 'SurveyPlan AI' folder or Root.")

In [ ]:
# 4. Run Training
import sys

# Define where to save results (Back to Drive)
RESULTS_DIR = f"{DRIVE_ROOT}/SurveyPlan AI/runs"
if not os.path.exists(RESULTS_DIR):
    os.makedirs(RESULTS_DIR)

print(f"\ud83d\ude80 Starting Training... Results will be saved to: {RESULTS_DIR}")

# Run src/train.py
# Note: We don't rely on 'runs' inside the VM. We let YOLO save there, then copy back.
# Or we could modify config. But copying is safer for now.

script_path = f"{CODE_DIR}/yolo_training/src/detection/train.py"

# We set PYTHONPATH so the script can 'import src...'
!PYTHONPATH="{CODE_DIR}/yolo_training" python "{script_path}" --data_path /content/custom_data

# 5. Backup Results
print(f"\ud83d\udcbe Backing up runs to Drive...")
vm_runs = "/content/runs"
if os.path.exists(vm_runs):
    # Copy new runs to Drive
    !cp -r "{vm_runs}/." "{RESULTS_DIR}/"
    print("\u2705 Backup complete!")
else:
    print("\u26a0\ufe0f No runs folder found to backup.")

In [ ]:
# 5. SAHI Sliced Inference

print("\ud83d\ude80 Starting SAHI Sliced Inference safely...")

# Install SAHI and Ultralytics if not already present
!pip install sahi ultralytics

inference_script = f"{CODE_DIR}/yolo_training/src/detection/inference.py"

# Use the best model from previous training, or fallback to the one in Drive if desired
# Assuming training saved to 'runs/obb/survey_plan_obb_run/weights/best.pt'
# Adjust path based on actual training output structure
best_model = f"{vm_runs}/obb/survey_plan_obb_run/weights/best.pt"

# If training didn't run or failed, check Drive for a pre-existing model
if not os.path.exists(best_model):
    print("\u26a0\ufe0f Training output not found. Checking Drive for 'best.pt'...")
    drive_model = f"{RESULTS_DIR}/obb/survey_plan_obb_run/weights/best.pt"
    if os.path.exists(drive_model):
        print(f"\u2705 Found model in Drive: {drive_model}")
        best_model = drive_model
    else:
        # Fallback to a default if you have one, or user must supply it
        print("\u274c No trained model found to run inference.")
        best_model = None

if best_model:
    inference_output = f"{RESULTS_DIR}/sahi_predict"
    print(f"\ud83d\udd0e Running inference on validation images... Output: {inference_output}")

    # Run inference on a sample set (e.g., validation images)
    # Check common locations for validation data (dataset.py moves them to data/)
    search_paths = [
        "/content/data/validation/images",
        "/content/custom_data/validation/images",
        "/content/custom_data"
    ]
    val_images = "/content/custom_data" # default fallback
    for p in search_paths:
        if os.path.exists(p) and any(os.scandir(p)):
            val_images = p
            break


    !PYTHONPATH="{CODE_DIR}/yolo_training" python "{inference_script}" --model_path "{best_model}" --source "{val_images}" --output_dir "{inference_output}"
    
    print(f"\u2705 Inference complete! Check {inference_output} in your Drive.")
else:
    print("\u23ed\ufe0f Skipping inference due to missing model.")